In [1]:
# -*- coding: utf-8 -*-
#%% NumPyの読み込み
import numpy as np
#   SciPyのstatsモジュールの読み込み
import scipy.stats as st
#%% Black-Scholeの公式によるヨーロピアン・コールオプション価格の計算
S = 100.0
K = 100.0
r = 0.01
v = 0.20
T = 0.50
d1 = (np.log(S / K) + (r + 0.5 * v ** 2) * T) / (v * np.sqrt(T))
d2 = d1 - v * np.sqrt(T)
BS_Formula = S * st.norm.cdf(d1) - K * np.exp(-r * T) * st.norm.cdf(d2)

In [3]:
print(BS_Formula)

5.876024233827607


# 20250206に更新

ブラックショールズ方程式は次のように表されます：

$$
C = S_0 N(d_1) - K e^{-rT} N(d_2)
$$

ここで：
- $ C $ はコールオプションの価格
- $ S_0 $ は現在の株価
- $ K $ は行使価格
- $ T $ はオプションの満期までの期間（年単位）
- $ r $ はリスクフリーレート（通常、国債利回りなど）
- $ N(d_1) $ と $ N(d_2) $ は累積分布関数（CDF）であり、通常は標準正規分布を使用
- $ \sigma $ は株価のボラティリティ

ブラックショールズ方程式の $ d_1 $ と $ d_2 $ は次のように計算されます：

$$
d_1 = \frac{\ln\left(\frac{S_0}{K}\right) + \left(r + \frac{\sigma^2}{2}\right) T}{\sigma \sqrt{T}}
$$

$$
d_2 = d_1 - \sigma \sqrt{T}
$$


In [1]:
import yfinance as yf
import numpy as np
import scipy.stats as si
from datetime import datetime

# ブラックショールズ方程式の実装
def black_scholes_call(S0, K, T, r, sigma):
    d1 = (np.log(S0 / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    call_price = S0 * si.norm.cdf(d1) - K * np.exp(-r * T) * si.norm.cdf(d2)
    return call_price

# 株式データの取得
ticker = 'AAPL'  # Apple株の例
stock = yf.Ticker(ticker)

# 株価データ（過去60日間）
history = stock.history(period="60d")
current_price = history['Close'][-1]  # 最新の株価

# パラメータ設定
K = 150  # 行使価格（仮）
T = 30 / 365  # 30日後の満期
r = 0.01  # リスクフリーレート（仮に1%）
sigma = 0.2  # ボラティリティ（仮に20%）

# コールオプション価格の計算
call_price = black_scholes_call(current_price, K, T, r, sigma)
print(f"現在の株価: {current_price}")
print(f"コールオプションの理論価格: {call_price}")


現在の株価: 232.47000122070312
コールオプションの理論価格: 82.59323823964803


C:\Users\20222305\AppData\Local\Temp\ipykernel_19796\2700821072.py:19: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  current_price = history['Close'][-1]  # 最新の株価


1. 理論価格 vs 実際の市場価格

- 理論価格（ブラックショールズ方程式による）は、主に株価、ボラティリティ、リスクフリーレート、行使価格、満期までの期間をもとに算出されます。
- 実際の市場価格は、投資家の需要と供給、マーケットセンチメント、流動性、時間的価値、金利、インフレ期待、その他のファンダメンタルズなど、さまざまな要因に影響を受けます。


2. オプションの割安・割高を判断する方法
- 割安（低価格）：理論価格が市場価格より高い場合、オプションは割安と判断できます。これは、市場で過小評価されている可能性を示唆します。
- 割高（高価格）：理論価格が市場価格より低い場合、オプションは割高と判断できます。これは、市場が過大評価している可能性を示唆します。

3. 買い時・売り時の判断
- 買い時：オプションが割安であると感じた場合、特に市場のボラティリティが低い状態であれば、買いのタイミングと考えられます。
- 売り時：逆にオプションが割高である場合、特に価格が過剰に膨らんでいると感じるなら、売りのタイミングかもしれません。

In [3]:
# 実際のオプション価格を取得する方法（例：AAPLのオプションチェーン）
options_chain = stock.options  # オプションチェーンの取得
expiration_date = options_chain[0]  # 一つ目の満期日を選択（例）
option_data = stock.option_chain(expiration_date)

# コールオプションのデータを表示
calls = option_data.calls
display(calls)

,contractSymbol,lastTradeDate,strike,lastPrice,bid,ask,change,percentChange,volume,openInterest,impliedVolatility,inTheMoney,contractSize,currency
0,AAPL250207C00100000,2025-02-03 20:20:30+00:00,100.0,128.70,131.70,133.30,0.0,0.0,4.0,32,4.015630,True,REGULAR,USD
1,AAPL250207C00110000,2025-01-31 19:41:43+00:00,110.0,124.44,121.85,123.30,0.0,0.0,1.0,11,4.093755,True,REGULAR,USD
2,AAPL250207C00120000,2025-01-29 19:56:22+00:00,120.0,118.30,111.70,113.30,0.0,0.0,NaN,4,3.187502,True,REGULAR,USD
3,AAPL250207C00130000,2025-02-04 16:55:44+00:00,130.0,102.00,101.90,103.30,0.0,0.0,1.0,1,3.328127,True,REGULAR,USD
4,AAPL250207C00140000,2025-02-04 14:30:07+00:00,140.0,87.17,91.90,93.30,0.0,0.0,1.0,3,2.937503,True,REGULAR,USD
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56,AAPL250207C00310000,2025-02-03 19:57:16+00:00,310.0,0.01,0.00,0.00,0.0,0.0,9.0,1067,0.500005,False,REGULAR,USD
57,AAPL250207C00315000,2025-02-04 15:24:09+00:00,315.0,0.01,0.00,0.00,0.0,0.0,1.0,200,0.500005,False,REGULAR,USD
58,AAPL250207C00320000,2025-01-30 20:25:43+00:00,320.0,0.01,0.00,0.00,0.0,0.0,12.0,150,0.500005,False,REGULAR,USD
59,AAPL250207C00325000,2025-01-30 20:45:54+00:00,325.0,0.01,0.00,0.01,0.0,0.0,255.0,313,1.406253,False,REGULAR,USD
